In [ ]:
!pip install -e .

Obtaining file:///home/christopher/Delphi_Cluster_Co_Activations
  Installing build dependencies ... done
  Checking if build backend supports build_editable ... done
  Getting requirements to build editable ... done
  Preparing editable metadata (pyproject.toml) ... done
  Building editable for co-activation-grouping (pyproject.toml) ... done
  Created wheel for co-activation-grouping: filename=co_activation_grouping-0.1.0-0.editable-py3-none-any.whl size=1643 sha256=3253f4c505be96818cd06fbb5b1fd0a39228756e0178dd24ba5a3d6a1bd8036b
  Stored in directory: /tmp/pip-ephem-wheel-cache-qrftw465/wheels/f5/74/8e/686a2059aa64e124473ac74d1192759e243186304c6a2d28f1
Successfully built co-activation-grouping
  Attempting uninstall: co-activation-grouping
    Found existing installation: co-activation-grouping 0.1.0
    Uninstalling co-activation-grouping-0.1.0:
      Successfully uninstalled co-activation-grouping-0.1.0


In [ ]:
from delphi.config import ConstructorConfig, SamplerConfig
from delphi.latents import (
    ActivatingExample,
    LatentDataset,
    LatentRecord,
)
import argparse
import torch

/home/christopher/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
model_name = "llama-3-8B"
latent_dict = {
  "layers.5.mlp": torch.arange(0, 100)
}
sampler_cfg = SamplerConfig()
constructor_cfg = ConstructorConfig()
# Delphi_Cluster_Co_Activations/src/data/llama-3-8B change this to raw dir?
dataset = LatentDataset(
    raw_dir="src/data/llama-3-8B/latents",
    modules=["layers.5.mlp"], # This a list of the different caches to load from
    sampler_cfg=sampler_cfg,
    constructor_cfg=constructor_cfg,
    latents=latent_dict,
    tokenizer=None
)

NameError: name 'torch' is not defined

In [ ]:
async for record in dataset:
  record

Not enough examples to explain the latent: 36
Not enough examples to explain the latent: 108
Not enough examples to explain the latent: 21
Not enough examples to explain the latent: 191
Not enough examples to explain the latent: 163
Not enough examples to explain the latent: 59
Not enough examples to explain the latent: 109
Not enough examples to explain the latent: 49
Not enough examples to explain the latent: 33
Not enough examples to explain the latent: 124
Not enough examples to explain the latent: 11
Not enough examples to explain the latent: 139
Not enough examples to explain the latent: 93
Not enough examples to explain the latent: 126
Not enough examples to explain the latent: 1
Not enough examples to explain the latent: 107
Not enough examples to explain the latent: 138
Not enough examples to explain the latent: 122
Not enough examples to explain the latent: 40


In [ ]:
buffer = dataset.buffers[0]

# Load raw data for all latents in this buffer
latents, split_locations, split_activations = buffer.load_data_per_latent()

In [ ]:
split_locations[32].size()

torch.Size([415036, 3])

In [ ]:
import numpy as np
from collections import defaultdict
from tqdm import tqdm

In [ ]:
latent_to_idx = {int(latent): idx for idx, latent in enumerate(latents)}
n_latents = len(latent_to_idx)
    
coactivation_matrix = np.zeros((n_latents, n_latents), dtype=int)
position_to_latents = defaultdict(set)

In [ ]:
for latent_locations in split_locations:
  
  for location in latent_locations:
    batch_idx = int(location[0])     
    token_idx = int(location[1])
    position_key = (batch_idx, token_idx) # position batch and token 
    
    latent = int(location[2])
    position_to_latents[position_key].add(latent) # adds latent to activate on the specific token

In [ ]:
for position, latents_at_position in tqdm(position_to_latents.items()):
  latent_list_at_pos = list(latents_at_position)
  
  for latent_i in latent_list_at_pos:
    index_i = latent_to_idx[latent_i] # converts the latent to index can be different as some latents are blank
    for latent_j in latent_list_at_pos:
        index_j =  latent_to_idx[latent_j]
        if latent_i != latent_j:
          coactivation_matrix[index_i, index_j] += 1

100%|██████████| 1377056/1377056 [00:01<00:00, 1187341.67it/s]


In [ ]:
coactivation_matrix

array([[  0,  86,  90, ...,  90,  90,  88],
       [ 86,   0, 448, ..., 448, 446, 446],
       [ 90, 448,   0, ..., 520, 739, 771],
       ...,
       [ 90, 448, 520, ...,   0, 508, 508],
       [ 90, 446, 739, ..., 508,   0, 506],
       [ 88, 446, 771, ..., 508, 506,   0]], shape=(98, 98))

In [ ]:
!pip install plotly "nbformat>=4.2.0"

In [ ]:
import plotly.express as px

In [ ]:
fig = px.imshow(coactivation_matrix,
  x=list(latent_to_idx.keys()),   # tick labels on x
  y=list(latent_to_idx.keys()),   # tick labels on y
  labels=dict(x="latent", y="latent"),
  title="Co-occurrence",
  color_continuous_scale="viridis")
# set colorbar title and nicer hover text
fig.update_coloraxes(colorbar_title="Co-occurrence frequency")
fig.data[0].hovertemplate = "x=%{x}<br>y=%{y}<br>co-occurrence=%{z}<extra></extra>"


# Display the figure
fig.show(renderer="browser")

In [ ]:
# import os

# directory_path = "/mnt/ssd-1/soar-automated_interpretability/graphs/pawan/results/gemmma-scope-2b-pt-transcoders" # Replace with your desired directory
# entries = os.listdir(directory_path)


# latent_dict = {
#   layer: torch.arange(0, 10000) for layer in entries
# }
# sampler_cfg = SamplerConfig()
# constructor_cfg = ConstructorConfig()

# # Delphi_Cluster_Co_Activations/src/data/llama-3-8B change this to raw dir?
# dataset = LatentDataset(
#     raw_dir= "/mnt/ssd-1/soar-automated_interpretability/graphs/pawan/results/gemmma-scope-2b-pt-transcoders",
#     modules=entries, # This a list of the different caches to load from
#     sampler_cfg=sampler_cfg,
#     constructor_cfg=constructor_cfg,
#     latents=latent_dict,
#     tokenizer=None
# )
# import os
# from safetensors import safe_open

# directory_path = "/mnt/ssd-1/soar-automated_interpretability/graphs/pawan/results/gemmma-scope-2b-pt-transcoders"
# test_layer = "layers.6.mlp"
# layer_path = os.path.join(directory_path, test_layer)

# file_path = os.path.join(layer_path, "sae.safetensors")

# with safe_open(file_path, framework="pt") as f:
#     print("Tensor shapes:")
#     for key in f.keys():
#         tensor = f.get_tensor(key)
#         print(f"  {key}: {tensor.shape}")
    
#     # The number of features is typically the first dimension of W_dec
#     # or the output dimension of encoder.weight
#     W_dec = f.get_tensor("W_dec")
#     print(f"\n✅ Number of features: {W_dec.shape[0]}")